### CPU 환경

In [1]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          64
On-line CPU(s) list:             0-63
Thread(s) per core:              2
Core(s) per socket:              16
Socket(s):                       2
NUMA node(s):                    2
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) Platinum 8153 CPU @ 2.00GHz
Stepping:                        4
CPU MHz:                         2299.998
CPU max MHz:                     2800.0000
CPU min MHz:                     1000.0000
BogoMIPS:                        4000.00
Virtualization:                  VT-x
L1d cache:                       1 MiB
L1i cache:                       1 MiB
L2 cache:                        32 MiB
L3 cache:                   

### GPU 환경

In [2]:
!nvidia-smi

Wed Feb 17 07:32:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 450.51.05    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:37:00.0 Off |                    0 |
| N/A   75C    P0    34W /  70W |   3259MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:86:00.0 Off |                    0 |
| N/A   

### RAM 용량 확인

In [3]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:          125Gi       7.7Gi       5.9Gi        50Mi       111Gi       116Gi
Swap:         8.0Gi        86Mi       7.9Gi


### HDD 용량 확인

In [4]:
# 디스크 용량 확인
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev             63G     0   63G   0% /dev
tmpfs            13G  2.7M   13G   1% /run
/dev/sda2       549G  418G  104G  81% /
tmpfs            63G  8.0K   63G   1% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs            63G     0   63G   0% /sys/fs/cgroup
/dev/sda1       511M  7.8M  504M   2% /boot/efi
/dev/loop4       56M   56M     0 100% /snap/core18/1944
tmpfs            13G     0   13G   0% /run/user/1000
/dev/loop6       32M   32M     0 100% /snap/snapd/10707
/dev/loop0       70M   70M     0 100% /snap/lxd/19032
/dev/loop1       70M   70M     0 100% /snap/lxd/19188
/dev/sdb2       466G  240G  226G  52% /home/test/mount_folder
/dev/loop5       56M   56M     0 100% /snap/core18/1988
/dev/loop2       32M   32M     0 100% /snap/snapd/11036


### OS 환경

In [5]:
!cat /etc/os-release

NAME="Ubuntu"
VERSION="20.04.1 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04.1 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal


In [6]:
import sys
import os
import os.path as osp

import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob

import pytz
import datetime
import time

import sklearn

import torch
import torchvision

import torch.nn as nn
from torch.utils.data import DataLoader

import efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from torchvision import datasets
import torchvision.models as models

import egg_dataset as Dataset
import explain

In [7]:
print('python version:',sys.version)
print('pandas version:',pd.__version__)
print('numpy version:', np.__version__)

print('sklearn version:', sklearn.__version__)
print('torch version:', torch.__version__)
print('torchvision version:', torchvision.__version__)
print('albumentations version:', A.__version__)
print('EfficientNet version:', efficientnet_pytorch.__version__)

python version: 3.7.0 (default, Oct  9 2018, 10:31:47) 
[GCC 7.3.0]
pandas version: 1.1.5
numpy version: 1.19.2
sklearn version: 0.23.2
torch version: 1.7.1
torchvision version: 0.8.2
albumentations version: 0.5.2
EfficientNet version: 0.7.0


In [8]:
# 모델에 입력할 Input 이미지의 크기를 지정함.
IMG_WIDTH = 512
IMG_HEIGHT = 512

# 배치 사이즈 임의 지정
batch_size = 4

# 데이터를 읽어올 경로를 지정함.
data_dir='../Dataset/preprocessed/classification'
# 학습된 모델이 저장된 경로를 지정함.
ckpt_dir = "./Models/"

# 모델 이름
Model_name = "egg_classification"
# 모델을 연산할 장비 설정 (Docker환경에서 작동할 수 있도록 CPU에서 계산함)
device = torch.device('cpu')

## Train, Validation, Test 데이터 셋 확인

## 데이터 셋 (Dataset) 불러오기

In [9]:
# 이미지 transformation
test_compose=A.Compose([
    A.Resize(IMG_HEIGHT,IMG_WIDTH),
    ToTensorV2()
])

# Test set을 읽어옴
test_dataset=Dataset.EggDatasetBasic(os.path.join(data_dir, 'test'), transforms=test_compose)
test_dataloader=DataLoader(test_dataset, batch_size=batch_size, pin_memory=False, shuffle=False)

### 모델 불러오기 (Load)

In [10]:
# Model: EfficientNet_B7
model=EfficientNet.from_name(model_name='efficientnet-b7', num_classes=1)
model._fc=nn.Sequential(nn.Linear(2560, 1, bias=True))
# 학습된 모델 가중치 불러오기
checkpoint=torch.load(ckpt_dir + Model_name + '.pth', map_location=device)
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

### 모델 학습 전, ImageNet 데이터로 Pretrain 된 기본 모델 불러오기 (Load)

In [11]:
base_model=EfficientNet.from_pretrained(model_name='efficientnet-b7', num_classes=1)

Loaded pretrained weights for efficientnet-b7


## 모델 확인 (Summary)

In [12]:
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        64, 16, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        16, 64, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        64, 32, kernel_siz

---

## 테스트 (Test) 수행

### 1. ImageNet 데이터로 Pretrain 된 기본 모델 성능 평가

### 테스트 이미지 평가 (Predict)

In [13]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 시작 시간 : {nowDatetime}')

테스트 시작 시간 : 2021-02-17 16:32:26


In [14]:
since=time.time()

In [15]:
tot_labels=[]
tot_pred_labels=[]

# 네트워크를 evaluation 용으로 선언
model=model.eval()

model=model.to(device)
# test 이기 때문에 backprop 진행 x
with torch.no_grad():
    for i, (images, target) in tqdm(enumerate(test_dataloader)):
        
        images=images.to(device)
        target=target.to(device)

        output=model(images)

        output_index=torch.round(torch.sigmoid(output.view(-1)))

        tot_labels.extend(list(target.numpy()))
        tot_pred_labels.extend(list(output_index.view(-1).detach().numpy()))

69it [12:13, 10.63s/it]


In [16]:
tot_labels=[]
tot_pred_labels=[]

# 네트워크를 evaluation 용으로 선언
base_model=base_model.eval()

base_model=base_model.to(device)
# test 이기 때문에 backprop 진행 x
with torch.no_grad():
    for i, (images, target) in tqdm(enumerate(test_dataloader)):
        
        images=images.to(device)
        target=target.to(device)

        output=base_model(images)

        output_index=torch.round(torch.sigmoid(output.view(-1)))

        tot_labels.extend(list(target.numpy()))
        tot_pred_labels.extend(list(output_index.view(-1).detach().numpy()))

69it [11:05,  9.65s/it]


In [17]:
end=time.time()
print(f'테스트 소요 시간: {int((end-since)//60)}m {int((end-since)%60)}s')

테스트 소요 시간: 23m 19s


In [18]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 종료 시간 : {nowDatetime}')

테스트 종료 시간 : 2021-02-17 16:55:46


## F1-Score

In [19]:
result=explain.ShowResult(tot_labels, tot_pred_labels)

In [20]:
result.show_result()

Predicted   A+   A  All
True                   
A+          82  11   93
A          154  26  180
All        236  37  273


#-- Confusion Matrix for class A+

              Pred     
            Non A+   A+
True Non A+     26  154
     A+         11   82
F1-Score for class A+ : 0.498
-----------------------------------

#-- Confusion Matrix for class A

            Pred    
           Non A   A
True Non A    82  11
     A       154  26
F1-Score for class A : 0.240
-----------------------------------

#-- Final Macro F1-Score
( 0.498 + 0.240 ) / 2 = 0.3691


---

### 2. 기본 모델을 Fine Tuning (학습) 한 모델 성능 평가 (Transfer Learning)

### 테스트 이미지 평가 (Predict)

In [21]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 시작 시간 : {nowDatetime}')

테스트 시작 시간 : 2021-02-17 16:55:46


In [22]:
since=time.time()

In [23]:
tot_labels=[]
tot_pred_labels=[]

# 네트워크를 evaluation 용으로 선언
model=model.eval()

model=model.to(device)
# test 이기 때문에 backprop 진행 x
with torch.no_grad():
    for i, (images, target) in tqdm(enumerate(test_dataloader)):
        
        images=images.to(device)
        target=target.to(device)

        output=model(images)

        output_index=torch.round(torch.sigmoid(output.view(-1)))

        tot_labels.extend(list(target.numpy()))
        tot_pred_labels.extend(list(output_index.view(-1).detach().numpy()))

69it [10:54,  9.48s/it]


In [24]:
end=time.time()
print(f'테스트 소요 시간: {int((end-since)//60)}m {int((end-since)%60)}s')

테스트 소요 시간: 10m 54s


In [25]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 종료 시간 : {nowDatetime}')

테스트 종료 시간 : 2021-02-17 17:06:40


## F1-Score

In [26]:
result=explain.ShowResult(tot_labels, tot_pred_labels)

In [27]:
result.show_result()

Predicted   A+    A  All
True                    
A+          77   16   93
A           26  154  180
All        103  170  273


#-- Confusion Matrix for class A+

              Pred    
            Non A+  A+
True Non A+    154  26
     A+         16  77
F1-Score for class A+ : 0.786
-----------------------------------

#-- Confusion Matrix for class A

            Pred     
           Non A    A
True Non A    77   16
     A        26  154
F1-Score for class A : 0.880
-----------------------------------

#-- Final Macro F1-Score
( 0.786 + 0.880 ) / 2 = 0.8329
